In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto
import torch
import time
import pandas as pd, html
from user_info import get_user_info
import requests
from tqdm.notebook import tqdm
from user_info import get_user_data_search_embed
from job_info import fetch_jobs

/home/ubuntu/infoedge/llama-recipes/env/lib/python3.10/site-packages/huggingface_hub/utils/_runtime.py:184: UserWarning: Pydantic is installed but cannot be imported. Please check your installation. `huggingface_hub` will default to not using Pydantic. Error message: '{e}'
  warnings.warn(


In [2]:
# from vllm import SamplingParams, LLM

# sampling_params = SamplingParams(temperature=0, max_tokens=144)
# llm = LLM(model="mistralai/Mistral-7B-Instruct-v0.2")

In [3]:
# model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2",
#                                             load_in_8bit=True, 
#                                             torch_dtype=torch.float16,
#                                             device_map='auto')

# tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")

In [4]:
dataset = pd.read_csv('data/analysis_dataset.csv')

In [5]:
dataset

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [6]:
def fetch_job_title(job_id):
    
    '''
    Returns Job Data[Hirst, Source : ElasticSearch] for the last_k days, and with flexible page size (<10,000)
    
    Input : Last-K Days, and Page result size
    
    Output : Pandas Dataframe with jobs from the last k days 
    '''
    
    # columns = ['id', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags','brandJobFlag']     #Added column name for companyStatus
    # new_cols = ['jobid', 'title','companyStatus', 'min', 'max', 'premium', 'applyCount', 'locations','tags', 'brandJobFlag']       #Added column name for companyStatus
    
    url = "http://internal-java-job-searcher-email-backend-1607736061.ap-south-1.elb.amazonaws.com/v1/user/-777/job/jobcode?query=<JOBID>"
    url = url.replace('<JOBID>',job_id)
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    
    payload={}
    headers = {}
    
    response = requests.request("GET", url, headers=headers, data=payload)
    job_data = response.json()

    title = job_data[0]['title']
    jd = job_data[0]['introText']
    jd = html.unescape(jd)

    return title + '\n' + jd


In [7]:
dataset

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933
...,...,...,...
136248,1376058,144442227,2151474
136249,1375030,144442249,1463255
136250,1373682,144442251,2601930
136251,1376582,144442257,1417492


In [8]:
dataset['job_id'].drop_duplicates()

0         1373681
25        1373682
66        1373683
71        1373684
189       1373685
           ...   
133468    1376446
133579    1376758
134063    1375146
136145    1376759
136172    1376737
Name: job_id, Length: 2949, dtype: int64

In [9]:
dataset['user_id'].drop_duplicates()

0          439042
1         2015063
2         2777927
3           58130
4         2290933
           ...   
136201    2796578
136202    2679806
136235     986712
136245    1713212
136249    1463255
Name: user_id, Length: 30363, dtype: int64

In [10]:
dataset.sample(5)

,job_id,application_id,user_id
34019,1374360,144325489,1177085
74348,1375541,144321437,1083622
130839,1376179,144432509,2744235
16637,1373909,144172562,1970935
60244,1374946,144267077,1712512


In [11]:
es_output = get_user_info(1358359)
rt = es_output['resume'][0]
rt = html.unescape(rt)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 15947.92it/s]


In [12]:
answer_format = {
    "work_experience":"description of most recent work experience"
}

In [13]:
input_prompt = f'''
<s>[INST]You are an accurate AI agent working for a job platform. You will be given the raw 
unstructured text of a user's resume, Your task is to return the description of the user's
latest job experience as given in the resume. return the answer in the following format :\n{{answer_format}}\n
Return a valid JSON only
This is the resume text:\n{{resume_text}}\n
The answer is: [\INST]
'''

In [35]:
input_prompt = input_prompt.format(resume_text=rt, answer_format=answer_format)

In [36]:
outputs = llm.generate(input_prompt, sampling_params)
generated_text = outputs[0].outputs[0].text

Processed prompts: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.07s/it]


In [37]:
generated_text

"{'work_experience': 'Currently working as a Process Specialist at Cargill since June 2021. Responsibilities include preparing Financial Statements in accordance with USGAAP and Cargill's internal guidelines, analyzing anomalies, working on periodical forecast and budget for expenses by cost centre wise, analyzing actual financial results compared with forecast and budget on monthly and quarterly basis, and providing data with trends to the commercial team. Additionally, I meet regulatory requirements by providing data to government authorities and provide cost drivers for changes in cost over months and drive monthly divisional cost governance meetings. Value-added projects include price, discount & cost analysis for effective maintenance of"

In [17]:
# start_time = time.time()
# model.eval()
# with torch.no_grad():
#     full_document = tokenizer.decode(model.generate(**model_input, max_new_tokens=32)[0], skip_special_tokens=True)

# print(f'Time taken : {time.time()-start_time}')

In [18]:
dataset.head(5)

,job_id,application_id,user_id
0,1373681,144274567,439042
1,1373681,144261479,2015063
2,1373681,144253966,2777927
3,1373681,144241259,58130
4,1373681,144236812,2290933


In [19]:
dataset.sample(frac=1)

,job_id,application_id,user_id
36786,1374412,144220143,158664
36286,1374410,144228411,1853933
32756,1374332,144230299,2793538
59461,1374912,144286093,849265
62427,1375002,144301729,84463
...,...,...,...
101756,1376152,144379743,1693198
125367,1376100,144422162,1268621
3177,1373739,144168429,1177512
31885,1374309,144216364,1702935


In [14]:
f = []
for jid in tqdm(dataset.job_id.unique()):
    sdf = fetch_jobs(str(jid))
    f.append(sdf)

jdf = pd.concat(f)

  0%|          | 0/2949 [00:00<?, ?it/s]

/tmp/ipykernel_4972/3556400985.py:6: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  jdf = pd.concat(f)


In [21]:
def get_answer_from_vllm(resume_text, job_title, llm):

    #define the input prompt here
    input_prompt = f'''<s>[INST]You are an accurate AI agent working for a job platform. You will be given the raw 
                            unstructured text of a user's resume, and the description of a job that they have applied to. Your task 
                            is to check if the resume contains prior work experience relevant to the job title being provided.
                            Respond with a JUST A SINGLE WORD answer "yes" or "no". If there's any ambiguity, answer with "no"
                            This is the job description {{job_title}}\n
                            This is the resume text:\n{{resume_text}}\n
                            The answer is: [\INST]
                            '''

    input_prompt = input_prompt.format(resume_text=resume_text,
                   job_title=jt)
    
    outputs = llm.generate(input_prompt, sampling_params)
    generated_text = outputs[0].outputs[0].text

    return generated_text

In [15]:
jdf.sample()

,id,title,introText,jobdesignation,min,max,createdBy,creatorDomainName,categoryId,jobDetailUrl,...,maxBatch,brandJobFlag,companyDomain,lableId,companyData,recruiter,jobStatusInfo,refreshStatus,showRecruiterProfile,location
0,1374165,Branch Manager - Agency Channel - Insurance (6...,<p>Duties & Responsibilities:<br/><br/>Respons...,Branch Manager - Agency Channel - Insurance,6,8,69500,servingskill.com,14,https://www.iimjobs.com/j/branch-manager-agenc...,...,None,0,None,None,"{'companyId': 0, 'companyName': 'Serving Skill...","{'recruiterId': 69500, 'recruiterName': 'Priti...",None,1,NaN,"[{'id': 2, 'name': 'Mumbai'}]"


In [16]:
jdf[jdf['applyCount']>=70].sample()

,id,title,introText,jobdesignation,min,max,createdBy,creatorDomainName,categoryId,jobDetailUrl,...,maxBatch,brandJobFlag,companyDomain,lableId,companyData,recruiter,jobStatusInfo,refreshStatus,showRecruiterProfile,location
0,1376606,Manager/Senior Manager - Risk & Compliance - K...,<p>JD :<br/><br/>- Risk & Compliance Managemen...,Manager/Senior Manager,6,10,157696,huquo.com,13,https://www.iimjobs.com/j/managersenior-manage...,...,None,0,None,None,"{'companyId': 0, 'companyName': 'Huquo', 'comp...","{'recruiterId': 157696, 'recruiterName': 'Kish...",None,1,NaN,"[{'id': 37, 'name': 'Gurgaon/Gurugram'}]"


In [20]:
#filter out jobs, select ones with atleast 50 applications
count_df = dataset['job_id'].value_counts().reset_index()
jobs = count_df.loc[(count_df['count']>=50) & (count_df['count']<60)].sample(10)

In [22]:
jobs

,job_id,count
840,1375026,51
848,1374956,50
752,1373879,56
851,1374981,50
798,1374473,54
796,1376033,54
729,1375642,58
835,1373929,51
845,1373732,51
852,1373755,50


In [26]:
sample_set = dataset[dataset['job_id'].isin(jobs['job_id'])]
job_details = sample_set[['job_id','user_id']]

In [52]:
applicants = job_details['user_id'].unique().tolist()

In [53]:
from user_info import fetch_data_es

In [54]:
es_data = fetch_data_es(applicants)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 476/476 [00:03<00:00, 130.89it/s]


In [55]:
r[0][0]['id']

209995

In [56]:
resumes = {}

for data in tqdm(es_data):
    try:
        userid = data[0]['id']
        resumes[userid] = data[0]['resumeText']
    except:
        pass

  0%|          | 0/476 [00:00<?, ?it/s]

In [57]:
len(resumes.keys())

473

In [62]:
import random
random.sample(list(resumes.keys()),1)

[2808728]

In [65]:
import json
with open('data/test_user_resume.json','w') as f:
    json.dump(resumes,f)

In [66]:
import json
with open('data/test_user_resume.json','r') as f:
    check = json.load(f)

In [68]:
len(check.keys())

473